In [1]:
import wrds
db = wrds.Connection(wrds_username='etiennebruno')

Loading library list...
Done


In [2]:
db.list_libraries().sort()
db.list_libraries()

['aha',
 'aha_sample',
 'ahasamp',
 'audit',
 'auditsmp',
 'auditsmp_all',
 'bank',
 'blab',
 'block',
 'block_all',
 'boardex',
 'boardex_trial',
 'boardsmp',
 'bvd',
 'bvdsamp',
 'calcbench_trial',
 'calcbnch',
 'cboe',
 'cboe_all',
 'centris',
 'ciq',
 'ciqsamp',
 'ciqsamp_common',
 'ciqsamp_transcripts',
 'cisdm',
 'cisdmsmp',
 'clrvt',
 'clrvtsmp',
 'columnar',
 'comp',
 'comp_bank',
 'comp_bank_daily',
 'comp_execucomp',
 'comp_global',
 'comp_global_daily',
 'comp_na_annual_all',
 'comp_na_daily_all',
 'comp_na_monthly_all',
 'comp_segments_hist',
 'comp_segments_hist_daily',
 'compa',
 'compb',
 'compbd',
 'compdcur',
 'compg',
 'compgd',
 'comph',
 'compm',
 'compmcur',
 'compnad',
 'compsamp',
 'compsamp_snapshot',
 'compseg',
 'compsegd',
 'compsnap',
 'comscore',
 'contrib',
 'contrib_ceo_turnover',
 'contrib_char_returns',
 'contrib_general',
 'contrib_intangible_value',
 'contrib_kpss',
 'contrib_liva',
 'crsp',
 'crsp_a_ccm',
 'crsp_a_indexes',
 'crsp_a_stock',
 'crsp_a_

In [3]:
# List the tables of a given library§
db.list_tables(library='crsp')

['acti',
 'asia',
 'asib',
 'asic',
 'asio',
 'asix',
 'bmdebt',
 'bmheader',
 'bmpaymts',
 'bmquotes',
 'bmyield',
 'bndprt06',
 'bndprt12',
 'bxcalind',
 'bxdlyind',
 'bxmthind',
 'bxquotes',
 'bxyield',
 'cap',
 'ccm_lookup',
 'ccm_qvards',
 'ccmxpf_linktable',
 'ccmxpf_lnkhist',
 'ccmxpf_lnkrng',
 'ccmxpf_lnkused',
 'comphead',
 'comphist',
 'compmaster',
 'contact_info',
 'crsp_cik_map',
 'crsp_daily_data',
 'crsp_header',
 'crsp_monthly_data',
 'crsp_names',
 'crsp_portno_map',
 'crsp_ziman_daily_index',
 'crsp_ziman_monthly_index',
 'cs20yr',
 'cs5yr',
 'cs90d',
 'cst_hist',
 'daily_nav',
 'daily_nav_ret',
 'daily_returns',
 'dividends',
 'dport1',
 'dport2',
 'dport3',
 'dport4',
 'dport5',
 'dport6',
 'dport7',
 'dport8',
 'dport9',
 'dsbc',
 'dsbo',
 'dse',
 'dse62',
 'dse62delist',
 'dse62dist',
 'dse62exchdates',
 'dse62names',
 'dse62nasdin',
 'dse62shares',
 'dseall',
 'dseall62',
 'dsedelist',
 'dsedist',
 'dseexchdates',
 'dsenames',
 'dsenasdin',
 'dseshares',
 'dsf',


In [4]:
# Get data
daily = db.get_table(library='crsp', table='fund_summary2', obs=10)
daily

,summary_period2,crsp_fundno,caldt,summary_period,nav_latest,nav_latest_dt,tna_latest,tna_latest_dt,yield,div_ytd,...,accrual_fund,sales_restrict,wbrger_obj_cd,policy,lipper_class,lipper_class_name,lipper_obj_cd,lipper_obj_name,lipper_asset_cd,lipper_tax_cd
0,A,1.0,1997-12-31,A,15.30,1997-12-31,81.0,1997-12-31,0.062007,0.948700,...,None,None,None,None,None,None,None,None,None,None
1,A,1.0,1998-12-31,A,15.07,1998-12-31,196.7,1998-12-31,0.067384,1.015472,...,None,None,None,None,None,None,BBB,Corporate Debt Funds BBB-Rated,None,None
2,Q,1.0,1999-03-31,Q,14.84,1999-03-31,217.2,1999-03-31,0.016527,0.245256,...,None,None,None,None,None,None,BBB,Corporate Debt Funds BBB-Rated,None,None
3,Q,1.0,1999-06-30,Q,14.40,1999-06-30,213.1,1999-06-30,0.034137,0.491576,...,None,None,None,None,None,None,BBB,Corporate Debt Funds BBB-Rated,None,None
4,Q,1.0,1999-09-30,Q,14.18,1999-09-30,208.4,1999-09-30,0.051997,0.737319,...,None,None,None,None,None,None,BBB,Corporate Debt Funds BBB-Rated,None,None
5,AQ,1.0,1999-12-31,Q,13.97,1999-12-31,200.1,1999-12-31,0.070372,0.983104,...,Y,Y,None,None,BBB,Corporate Debt Funds BBB-Rated,BBB,Corporate Debt Funds BBB-Rated,TX,Taxable
6,Q,1.0,2000-03-31,Q,13.89,2000-03-31,185.9,2000-03-31,0.017938,0.249155,...,Y,Y,None,None,BBB,Corporate Debt Funds BBB-Rated,BBB,Corporate Debt Funds BBB-Rated,TX,Taxable
7,Q,1.0,2000-06-30,Q,13.81,2000-06-30,182.1,2000-06-30,0.034981,0.483094,...,Y,Y,None,None,BBB,Corporate Debt Funds BBB-Rated,BBB,Corporate Debt Funds BBB-Rated,TX,Taxable
8,Q,1.0,2000-09-29,Q,12.19,2000-07-31,183.7,2000-07-31,0.044742,0.545400,...,None,None,None,None,None,None,None,None,None,None
9,A,2.0,1997-12-31,A,17.09,1997-12-31,72.0,1997-12-31,0.018724,0.320000,...,None,None,None,None,None,None,None,None,None,None


In [5]:
for table in db.list_tables(library='crsp'):
    try:
        l= list(filter(lambda x: 'fund_name' in x, db.get_table(library='crsp', table=table, obs=1).keys()))
    except:
        pass
    if len(l) > 0:
        print(table, "---", l)

fund_hdr --- ['fund_name']
fund_hdr_hist --- ['fund_name']
fund_names --- ['fund_name']
fund_summary2 --- ['fund_name']
mfdbname --- ['fund_name']
portnomap --- ['fund_name']


In [31]:
%%time
df_funds = db.get_table(library='crsp', table='portnomap', obs=5)
df_funds

CPU times: user 2.46 ms, sys: 756 µs, total: 3.22 ms
Wall time: 96.9 ms


,crsp_fundno,crsp_portno,begdt,enddt,cusip8,crsp_cl_grp,fund_name,ticker,ncusip,first_offer_dt,...,retail_fund,inst_fund,m_fund,index_fund_flag,vau_fund,et_flag,end_dt,dead_flag,delist_cd,merge_fundno
0,4273.0,1000001.0,2003-07-31,2010-05-31,01877E10,2000866.0,"AB Cap Fund, Inc: AB Small Cap Growth Portfoli...",QUASX,01877E107,1969-02-12,...,Y,N,N,None,N,None,2021-12-31,N,None,NaN
1,4274.0,1000001.0,2003-07-31,2010-05-31,01877E20,2000866.0,"AB Cap Fund, Inc: AB Small Cap Growth Portfoli...",QUABX,01877E206,1990-09-17,...,Y,N,N,None,N,None,2019-10-31,Y,M,4273.0
2,4275.0,1000001.0,2003-07-31,2010-05-31,01877E40,2000866.0,"AB Cap Fund, Inc: AB Small Cap Growth Portfoli...",QUACX,01877E404,1993-05-03,...,Y,N,N,None,N,None,2021-12-31,N,None,NaN
3,4276.0,1000001.0,2003-07-31,2010-05-31,01877E50,2000866.0,"AB Cap Fund, Inc: AB Small Cap Growth Portfoli...",QUAYX,01877E503,1996-10-01,...,N,Y,N,None,N,None,2021-12-31,N,None,NaN
4,4277.0,1000001.0,2005-03-31,2010-05-31,01877E60,2000866.0,"AB Cap Fund, Inc: AB Small Cap Growth Portfoli...",QUARX,01877E602,2005-03-01,...,N,Y,N,None,N,None,2021-12-31,N,None,NaN


In [41]:
df_funds_map = db.raw_sql('''
        select distinct on (fund_name) fund_name, ticker, ncusip, cusip8
        from crsp.portnomap
        where begdt <= '2020-01-01' --and enddt >= '2022-01-01'
        order by fund_name asc
''')

In [42]:
df_funds_map

,fund_name,ticker,ncusip,cusip8
0,1290 Funds: 1290 Convertible Securities Fund; ...,TNFAX,68246A793,68246A79
1,1290 Funds: 1290 Convertible Securities Fund; ...,TNFIX,68246A777,68246A77
2,1290 Funds: 1290 Convertible Securities Fund; ...,TNFRX,68246A769,68246A76
3,1290 Funds: 1290 Diversified Bond Fund; Class ...,TNUAX,68246A710,68246A71
4,1290 Funds: 1290 Diversified Bond Fund; Class ...,TNUIX,68246A686,68246A68
...,...,...,...,...
53857,YieldQuest Funds Trust: YieldQuest Tax-Exempt ...,YQTEX,98584A303,98584A30
53858,YieldQuest Funds Trust: YieldQuest Tax-Exempt ...,YTEIX,98584A881,98584A88
53859,YieldQuest Funds Trust: YieldQuest Total Retur...,YQTRX,98584A204,98584A20
53860,YieldQuest Funds Trust: YieldQuest Total Retur...,YTRIX,98584A808,98584A80


In [44]:
db.raw_sql('''
        select distinct fund_name
        from crsp.portnomap
        where lower(fund_name::text) LIKE '%%s&p%%'
            and lower(fund_name::text) LIKE '%%500%%'
            --and lower(fund_name::text) LIKE '%%bond%%'
            --and lower(fund_name::text) LIKE '%%national%%'
            --and caldt >= '2010-01-31'
     limit 100
''')

,fund_name
0,Advisors' Inner Circle Fund: Toews S&P 500 Hed...
1,Advisors' Inner Circle Fund: Toews S&P 500 Hed...
2,Advisors' Inner Circle Fund: United Associatio...
3,Advisors' Inner Circle Fund: United Associatio...
4,AIM Counselor Series Trust (Invesco Counselor ...
...,...
95,Global X Funds: Global X S&P 500 Quality Divid...
96,Global X Funds: Global X S&P 500 Risk Managed ...
97,Global X Funds: Global X S&P 500 Tail Risk ETF
98,"Great West Fund, Inc: Great West S&P 500 Index..."


In [66]:
db.raw_sql('''
        select *
        from crsp.fund_summary2
        where fund_name LIKE '%%FundVantage Trust: Gotham Enhanced S&P 500 Index Fund; Institutional Class Shares%%'
        --and caldt >= '2010-01-31'
''')

,summary_period2,crsp_fundno,caldt,summary_period,nav_latest,nav_latest_dt,tna_latest,tna_latest_dt,yield,div_ytd,...,accrual_fund,sales_restrict,wbrger_obj_cd,policy,lipper_class,lipper_class_name,lipper_obj_cd,lipper_obj_name,lipper_asset_cd,lipper_tax_cd
0,Q,90300.0,2017-03-31,Q,10.63,2017-03-31,2.1,2017-03-31,NaN,NaN,...,N,N,None,None,SPSP,S&P 500 Index Funds,SP,S&P 500 INDEX OBJECTIVE FUNDS,EQ,Taxable
1,Q,90300.0,2017-06-30,Q,10.80,2017-06-30,2.7,2017-06-30,NaN,NaN,...,N,N,None,None,SPSP,S&P 500 Index Funds,SP,S&P 500 INDEX OBJECTIVE FUNDS,EQ,Taxable
2,Q,90300.0,2017-09-29,Q,11.53,2017-09-29,2.9,2017-09-29,NaN,NaN,...,N,N,None,None,SPSP,S&P 500 Index Funds,SP,S&P 500 INDEX OBJECTIVE FUNDS,EQ,Taxable
3,AQ,90300.0,2017-12-29,Q,12.08,2017-12-29,3.2,2017-12-29,0.013083,0.158048,...,N,N,None,None,LCCE,Large-Cap Core Funds,G,GROWTH FUNDS,EQ,Taxable
4,Q,90300.0,2018-03-29,Q,12.16,2018-03-29,3.1,2018-03-29,NaN,NaN,...,N,N,None,None,LCCE,Large-Cap Core Funds,G,GROWTH FUNDS,EQ,Taxable
5,Q,90300.0,2018-06-29,Q,12.68,2018-06-29,3.3,2018-06-29,NaN,NaN,...,N,N,None,None,LCCE,Large-Cap Core Funds,G,GROWTH FUNDS,EQ,Taxable
6,Q,90300.0,2018-09-28,Q,13.70,2018-09-28,3.5,2018-09-28,NaN,NaN,...,N,N,None,None,LCCE,Large-Cap Core Funds,G,GROWTH FUNDS,EQ,Taxable
7,AQ,90300.0,2018-12-31,Q,9.98,2018-12-31,3.1,2018-12-31,0.020723,0.206817,...,N,N,None,None,LCCE,Large-Cap Core Funds,G,GROWTH FUNDS,EQ,Taxable
8,Q,90300.0,2019-03-29,Q,11.30,2019-03-29,3.5,2019-03-29,NaN,NaN,...,N,N,None,None,LCCE,Large-Cap Core Funds,G,GROWTH FUNDS,EQ,Taxable
9,Q,90300.0,2019-06-28,Q,11.64,2019-06-28,3.7,2019-06-28,NaN,NaN,...,N,N,None,None,LCCE,Large-Cap Core Funds,G,GROWTH FUNDS,EQ,Taxable


In [64]:
db.raw_sql('''
        select *
        from crsp.portnomap
        where (fund_name::text) LIKE '%%FundVantage Trust: Gotham Enhanced S&P 500 Index Fund; Institutional Class Shares%%'
        --and caldt >= '2010-01-31'
''')

,crsp_fundno,crsp_portno,begdt,enddt,cusip8,crsp_cl_grp,fund_name,ticker,ncusip,first_offer_dt,...,retail_fund,inst_fund,m_fund,index_fund_flag,vau_fund,et_flag,end_dt,dead_flag,delist_cd,merge_fundno
0,90300.0,1049570.0,2017-01-01,2021-12-31,36087532,2103780.0,FundVantage Trust: Gotham Enhanced S&P 500 Ind...,GSPFX,360875322,2016-12-30,...,N,Y,N,B,N,None,2021-12-31,N,None,None


In [76]:
db.raw_sql('''
        select crsp_fundno, count(mret), max(mret), avg(mret)
        from crsp_q_mutualfunds.monthly_tna_ret_nav
        group by crsp_fundno
        order by count(mret) desc
     limit 10
''')

,crsp_fundno,count,max,avg
0,11972.0,724,0.116549,0.008918
1,11893.0,722,0.182182,0.009561
2,12884.0,720,0.191419,0.007158
3,5254.0,720,0.134024,0.009214
4,30976.0,720,0.428390,0.010923
5,22030.0,720,0.183365,0.009411
6,12883.0,720,0.223485,0.007752
7,18675.0,720,0.147430,0.006090
8,24797.0,720,0.202020,0.009152
9,31276.0,720,0.186667,0.009923
